In [4]:
import cbor2
import sqlite3
import yaml
from monad_std import Option, std_types

In [2]:
fp = r'E:\WBH\Games\eve forever\EVE Data\sde\sde\fsd\blueprints.yaml'
with open(fp, 'r', encoding='utf-8') as f:
    blueprints_data = yaml.load(f, yaml.CLoader)
fp = r'E:\WBH\Games\eve forever\EVE Data\sde\sde\fsd\typeIDs.yaml'
with open(fp, 'r', encoding='utf-8') as f:
    type_data = yaml.load(f, yaml.CLoader)

In [5]:
blp_map = {}
for type_id, blp_data in blueprints_data.items():
    # prod_flag: 0 for Null, 1 for manufacture, 2 for reaction
    prod_flag = 0
    if 'activities' in blp_data.keys():
        if 'manufacturing' in blp_data['activities'].keys():
            prod_flag = 1
            materials = {}
            prod = {}
            time = 1
            if 'materials' in blp_data['activities']['manufacturing'].keys():
                for m in blp_data['activities']['manufacturing']['materials']:
                    materials[m['typeID']] = m['quantity']
            if 'products' in blp_data['activities']['manufacturing'].keys():
                for p in blp_data['activities']['manufacturing']['products']:
                    prod[p['typeID']] = p['quantity']
            if 'time' in blp_data['activities']['manufacturing'].keys():
                time = blp_data['activities']['manufacturing']['time']
        elif 'reaction' in blp_data['activities'].keys():
            prod_flag = 2
            materials = {}
            prod = {}
            if 'materials' in blp_data['activities']['reaction'].keys():
                for r in blp_data['activities']['reaction']['materials']:
                    materials[r['typeID']] = r['quantity']
            if 'products' in blp_data['activities']['reaction'].keys():
                for p in blp_data['activities']['reaction']['products']:
                    prod[p['typeID']] = p['quantity']
            if 'time' in blp_data['activities']['reaction'].keys():
                time = blp_data['activities']['reaction']['time']

    if prod_flag == 0:
        blp_map[type_id] = {'flag': 0, 'time': 0}
    else:
        blp_map[type_id] = {
            'flag': prod_flag,
            'material': materials,
            'product': Option.from_nullable(prod)
                .map(lambda v: std_types.MList((x, y) for x, y in v.items()))
                .and_then(lambda v: v.get(0)),
            'time': time
        }

In [6]:
db = sqlite3.connect("blueprints.db")
cursor = db.cursor()

In [13]:
cursor.execute("delete from material;")
db.commit()

In [7]:
_sql = """create table if not exists material (
    type_id integer primary key not null,
    flag integer not null default 0,
    material blob not null,
    product blob,
    time integer not null default 1
);"""
cursor.execute(_sql)
db.commit()

In [14]:
_sql = "insert into material values (?, ?, ?, ?, ?);"
_null_sql = "insert into material values (?, ?, ?, null, ?);"
for blp_id, blp_data in blp_map.items():
    if blp_data['flag'] == 0:
        mat_bin = cbor2.dumps({})
        prod_bin = Option.none()
    else:
        mat_bin = cbor2.dumps(blp_data['material'])
        prod_bin = blp_data['product'].map(cbor2.dumps)
    tm = blp_data['time']
    flag = blp_data['flag']
    if tm == 0:
        tm = 1
    if prod_bin.is_some():
        cursor.execute(_sql, (blp_id, flag, mat_bin, prod_bin.unwrap_unchecked(), tm))
    else:
        cursor.execute(_null_sql, (blp_id, flag, mat_bin, tm))
db.commit()

In [13]:
_sql = """create table if not exists blp_name (
    type_id integer primary key not null,
    type_name text not null
);"""
cursor.execute(_sql)
db.commit()

In [18]:
_sql = "insert into blp_name values (?, ?);"
for blp_id in blp_map.keys():
    name = None
    if 'name' in type_data[blp_id].keys():
        if 'zh' in type_data[blp_id]['name'].keys():
            name = type_data[blp_id]['name']['zh']
        elif 'en' in type_data[blp_id]['name'].keys():
            name = type_data[blp_id]['name']['en']
    if name is None:
        name = f"无名称物品（{blp_id}）"
    cursor.execute(_sql, (blp_id, name))
db.commit()